In [ ]:
from sqlalchemy import create_engine
import pandas as pd 
import numpy as np
mysql_setting = {
    'host' : "localhost",
    'dbname': "spider",
    'user': "root",
    'password': ""
}

def getDatabaseConn():
    engine = create_engine('mysql://{user}:{password}@{host}:3306/{dbname}'.format(**mysql_setting), echo=False)
    conn = engine.connect()
    return conn
conn = getDatabaseConn()

sql_lg_company = '''
    select full_name, short_name, size, industry, finance_stage from lagou_company  where company_id in (SELECT company_id  FROM spider.lagou_recruit_day where city = '上海' group by company_id, city)
'''
lg_company_df = pd.read_sql(sql_lg_company, conn)

def getCompanyFilter(line):
    try:
        import re
        p = re.compile(r'([\d]+).*')
        return pd.Series([int(p.findall(line)[0])])
    except Exception as e:
        return pd.Series([0])
lg_company_df_deal = pd.concat([lg_company_df, lg_company_df['size'].apply(getCompanyFilter)], axis=1).rename(columns={0:'low_size'})
lg_company_df_deal[lg_company_df_deal.low_size >= 500].sort_values('low_size').to_excel('/Users/monstar/Desktop/lagou_company.xlsx', index=False, index_label=False)
sql_zl_company = '''
    select full_name, size, company_nature, website, industry, address, company_url from zhilian_company where company_md5 in (SELECT distinct(company_md5) FROM spider.zhilian_position where city = '上海')
'''
zl_company_df = pd.read_sql(sql_zl_company, conn)
zl_company_df = pd.concat([zl_company_df, zl_company_df['size'].apply(getCompanyFilter)], axis=1).rename(columns={0:'low_size'})
zl_company_df[zl_company_df.low_size >= 500].sort_values('low_size').to_excel('/Users/monstar/Desktop/zhilian_company.xlsx', index=False, index_label=False)